# Lab 2 PySpark:

### Machine Learning

#### But first, let's do some SQL :)

**First, install and import PySPark and SparkSession**

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 47.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=291ba5a866517e49f306bd9f726288a0be880412dde834a53e0185288de375d9
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.15
Branch HEAD
Compiled by user ubuntu on 2022-06-09T19:58:58Z
Revision f74867bddfbcdd4d08076db36851e88b15e66556
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

**Now, download the dataset**

In [4]:
!gdown https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_ -O PatientInfo.csv

Downloading...
From: https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_
To: /content/PatientInfo.csv
100% 489k/489k [00:00<00:00, 109MB/s]


**Create a SparkSession object and name the app "Lab2"**

In [5]:
from pyspark.sql import SparkSession
ss = SparkSession.builder.appName("Lab2-G2")

In [6]:
spark = SparkSession.builder.getOrCreate()

**1. Read the file PatientInfo.csv into a dataframe**

In [7]:
patient = spark.read.option("header", True).csv('PatientInfo.csv')

**Show the first 5 lines of the dataframe**

In [8]:
patient.head(5)

[Row(patient_id='1000000001', sex='male', age='50s', country='Korea', province='Seoul', city='Gangseo-gu', infection_case='overseas inflow', infected_by=None, contact_number='75', symptom_onset_date='2020-01-22', confirmed_date='2020-01-23', released_date='2020-02-05', deceased_date=None, state='released'),
 Row(patient_id='1000000002', sex='male', age='30s', country='Korea', province='Seoul', city='Jungnang-gu', infection_case='overseas inflow', infected_by=None, contact_number='31', symptom_onset_date=None, confirmed_date='2020-01-30', released_date='2020-03-02', deceased_date=None, state='released'),
 Row(patient_id='1000000003', sex='male', age='50s', country='Korea', province='Seoul', city='Jongno-gu', infection_case='contact with patient', infected_by='2002000001', contact_number='17', symptom_onset_date=None, confirmed_date='2020-01-30', released_date='2020-02-19', deceased_date=None, state='released'),
 Row(patient_id='1000000004', sex='male', age='20s', country='Korea', provin

**Now do the same but using SQL select statement**

1. Create a temporary view (table) called patients

In [9]:
patient.createOrReplaceTempView("patient")

2. Use SELECT statement to select all columns from the dataframe

In [10]:
result = spark.sql("select * from patient")
result.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

3. Limit the output to only 5 rows *using SQL commands*

In [11]:
result = spark.sql("select * from patient limit 5")
result.show()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

4. Select the count of males and females in the dataset

In [12]:
result = spark.sql("select count(sex) as m from patient where sex = 'male' ")
result.show()
result = spark.sql("select count(sex) as f from patient where sex = 'female' ")
result.show()

+----+
|   m|
+----+
|1825|
+----+

+----+
|   f|
+----+
|2218|
+----+



In [13]:
result = spark.sql("select count(sex), sex from patient group by sex ")
result.show()

+----------+------+
|count(sex)|   sex|
+----------+------+
|         0|  null|
|      2218|female|
|      1825|  male|
+----------+------+



5. Select the count of males and females *as percentage* (how many percent of the data are males and how many are females?)

In [14]:
result = spark.sql("SELECT sex, (count(sex) * 100.0 / (SELECT count(sex) from patient)) as percentage FROM patient group by sex")
result.show()

+------+-----------------+
|   sex|       percentage|
+------+-----------------+
|  null|            0E-14|
|female|54.86025228790502|
|  male|45.13974771209498|
+------+-----------------+



6. How many people did survive, and how many didn't?

In [15]:
result = spark.sql("select count(state) from patient group by state")
result.show()

+------------+
|count(state)|
+------------+
|        2158|
|        2929|
|          78|
+------------+



Now, let's perform some preprocessing using SQL:

1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [16]:
import pyspark
from pyspark.sql.functions import col, substring
newDF = spark.sql('select sex, double(substring(age, 0,2)) as age, province, state from patient')
newDF.show()

+------+----+--------+--------+
|   sex| age|province|   state|
+------+----+--------+--------+
|  male|50.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|  male|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|20.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|female|60.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|80.0|   Seoul|deceased|
|female|60.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|70.0|   Seoul|released|
|female|70.0|   Seoul|released|
+------+----+--------+--------+
only showing top 20 rows



In [17]:
result = spark.sql("select * from patient")
result.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

Now view the new dataframe to make sure everything is alright

In [18]:
newDF.show()

+------+----+--------+--------+
|   sex| age|province|   state|
+------+----+--------+--------+
|  male|50.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|  male|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|20.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|female|60.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|80.0|   Seoul|deceased|
|female|60.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|70.0|   Seoul|released|
|female|70.0|   Seoul|released|
+------+----+--------+--------+
only showing top 20 rows



**Now, let's get back to spark operations**

Please copy the following operations from your solution in Lab 1
___

Add a **is_dead** column if patient state is not released then it should yield true, else then False

In [19]:
from pyspark.sql.functions import *
newDF = newDF.withColumn("is_dead",when(col("state") == 'released', 1).otherwise(0))
newDF.show()

+------+----+--------+--------+-------+
|   sex| age|province|   state|is_dead|
+------+----+--------+--------+-------+
|  male|50.0|   Seoul|released|      1|
|  male|30.0|   Seoul|released|      1|
|  male|50.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|female|20.0|   Seoul|released|      1|
|female|50.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|  male|30.0|   Seoul|released|      1|
|female|60.0|   Seoul|released|      1|
|female|50.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|  male|80.0|   Seoul|deceased|      0|
|female|60.0|   Seoul|released|      1|
|  male|70.0|   Seoul|released|      1|
|  male|70.0|   Seoul|released|      1|
|  male|70.0|   Seoul|released|      1|
|  male|20.0|   Seoul|released|      1|
|female|70.0|   Seoul|released|      1|
|female|70.0|   Seoul|released|      1|
+------+----+--------+--------+-------+
only showing top 20 rows



**Please split the data into train and test dataframes**

*Ratio: 80:20 - Seed=42*

In [20]:
trainDF, testDF = newDF.randomSplit(weights=[0.8,0.2], seed=42)

In [21]:
testDF.show(5)

+----+----+-----------+--------+-------+
| sex| age|   province|   state|is_dead|
+----+----+-----------+--------+-------+
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
|null|null|Gyeonggi-do|isolated|      0|
+----+----+-----------+--------+-------+
only showing top 5 rows



**Now, let's import RandomForestClassifier and start our ML pipeline**

**Create a pipeline that contains the following stages:**

- Imputer: impute the null values in `age` column to the mean value
- StringIndexer: convert `sex` to `is_male` and `province` to `province_index` as numerical values
- OneHotEncoder: perform one hot encoding on both `is_male` and -province_index`
- VectorAssembler: assemble feature vector from the following columns: `'age', 'is_male', 'province_index'`
- RandomForestClassifier: final estimator

In [22]:
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCol='age', outputCol='age', strategy = 'mean')

In [23]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCols=['sex','province'],outputCols = ['is_male1','province_index1'],handleInvalid = 'skip')

In [24]:
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderModel
onehotencoder = OneHotEncoder(inputCols=['is_male1','province_index1'], outputCols=["is_male","province_index"])

In [25]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=['age', 'is_male', 'province_index'],
    outputCol="features")


In [26]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features",labelCol="is_dead")

Fit the pipeline to the train dataframe

In [27]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[imputer, indexer, onehotencoder,assembler, rf])
pipeline_model = pipeline.fit(trainDF)


Now transform the test DF to get predictions

In [28]:
pred = pipeline_model.transform(testDF)

Show the final predictions DF

In [29]:
pred.show()

+------+-----------------+-----------------+--------+-------+--------+---------------+-------------+--------------+--------------------+--------------------+--------------------+----------+
|   sex|              age|         province|   state|is_dead|is_male1|province_index1|      is_male|province_index|            features|       rawPrediction|         probability|prediction|
+------+-----------------+-----------------+--------+-------+--------+---------------+-------------+--------------+--------------------+--------------------+--------------------+----------+
|female|40.83025210084033|Chungcheongnam-do|released|      1|     0.0|            4.0|(1,[0],[1.0])|(16,[4],[1.0])|(18,[0,1,6],[40.8...|[4.41741004721493...|[0.22087050236074...|       1.0|
|female|40.83025210084033|Chungcheongnam-do|released|      1|     0.0|            4.0|(1,[0],[1.0])|(16,[4],[1.0])|(18,[0,1,6],[40.8...|[4.41741004721493...|[0.22087050236074...|       1.0|
|female|40.83025210084033|            Daegu|isolat

In [30]:
pred.select('features','is_dead','prediction').show(5,truncate=False)

+----------------------------------------+-------+----------+
|features                                |is_dead|prediction|
+----------------------------------------+-------+----------+
|(18,[0,1,6],[40.83025210084033,1.0,1.0])|1      |1.0       |
|(18,[0,1,6],[40.83025210084033,1.0,1.0])|1      |1.0       |
|(18,[0,1,8],[40.83025210084033,1.0,1.0])|0      |0.0       |
|(18,[0,1,4],[40.83025210084033,1.0,1.0])|0      |0.0       |
|(18,[0,1,3],[40.83025210084033,1.0,1.0])|0      |1.0       |
+----------------------------------------+-------+----------+
only showing top 5 rows



**Model Evaluation**

Now let's evaluate our model! Let's get the accuracy of our model

In [37]:
from sklearn import metrics
metrics.accuracy_score(pred.select("is_dead").toPandas(), pred.select("prediction").toPandas())

0.8925925925925926

Excellent! Now let's generate the confusion matrix of our predictions

*Hint: we can use `scikit-learn`'s `classification_report`. You will need to transform the predictions into pandas DF first*

In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pred.select("is_dead").toPandas(), pred.select("prediction").toPandas())


array([[239,  67],
       [ 20, 484]])

## Great Job!

**If you followed the instructions correctly, you should get a total accuracy of 89%, and F1 scores of 92% and 85%**

**Do you think you can improve this accuracy? Let's see what you can do :)**

___
If you have any questions you can reach out to me:

### Omar Hammad
#### Software Engineer
##### Email: ommar365@gmail.com
##### Phone: 01144070145
##### Linkedin: https://www.linkedin.com/in/omar-a-hammad